# Reto 01 - Prompt Engineering

Consejo rápido: Para ver la Tabla de Contenidos de este Notebook en VS Code, en la pestaña "Explorer", expande la sección "Outline".

## 1. Experimentación de Parámetros
Primero configuraremos el Desafío. Carga la clave API y las bibliotecas de Python relevantes utilizando las celdas a continuación.

### 1.1 Preparación
#### 1.1.1 Configuración del Entorno


#### Crea o actualiza el archivo de entorno llamado ".env"
Hemos incluido un archivo de entorno de ejemplo para tu referencia. Puedes encontrar el archivo `.env.sample` en la raiz de este repositorio o en la carpeta `Student/Resources` dentro de la documentación del desafío. Siéntete libre de hacer las modificaciones necesarias y renombra el archivo como `.env`.

**NOTA:** Asegúrate de mantener el nombre de cada modelo igual al nombre de implementación del modelo que configuraste en Azure OpenAI.

**NOTA:** Ten en cuenta que algunos modelos de AOAI podrían estar obsoletos. Deberías crear los que estén disponibles según sea necesario. Para completar este primer desafío, gpt-35-turbo es el único modelo que necesitas.

Para obtener información sobre los modelos actuales, consulta: [Modelos del Servicio Azure OpenAI Service](https://learn.microsoft.com/es-mx/azure/ai-services/openai/concepts/models)

####  Configuración del Entorno de Azure OpenAI

In [3]:
import openai
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [4]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Explicación de Parámetros
Especificamos los hiperparámetros para los modelos de Azure OpenAI dentro de las funciones de ayuda. Los usuarios pueden ajustar los parámetros según sus necesidades.


###### **Temperatura**
La temperatura varía de 0 a 2. Aquí tienes un desglose rápido de cómo funciona:
- Temperatura baja (0 a 0.3): Salidas más enfocadas, coherentes y conservadoras.
- Temperatura media (0.3 a 0.7): Creatividad y coherencia equilibradas.
- Alta temperatura (> 0.7): Altamente creativa y diversa, pero potencialmente menos coherente.

###### **Top_p**
Establece el límite de probabilidad para el muestreo de tokens, afectando la amplitud de opciones que considera la IA. Valores más altos conducen a más aleatoriedad, mientras que valores más bajos resultan en salidas más enfocadas.


El parámetro "top_p" actúa como un filtro que controla cuántas palabras o frases diferentes considera el modelo de lenguaje cuando intenta predecir la siguiente palabra. Si configuras el valor de "top p" a 0.5, el modelo de lenguaje solo considerará las 50 palabras o frases más probables que podrían seguir. Pero si configuras el valor de "top p" a 0.9, el modelo de lenguaje considerará las 90 palabras o frases más probables.

Dado que "top_p" y "temperatura" realizan trabajos similares como hiperparámetros, usualmente solo ajustamos uno de ellos en lugar de ambos al mismo tiempo.

###### **Max_tokens**
Max_tokens determina la longitud máxima del texto generado. Al establecer un límite, puedes controlar cuánto texto devolverá el modelo LLM, asegurando que no proporcione una respuesta demasiado larga.

###### **Frequency_penalty**
La penalización de frecuencia asegura que el texto generado sea variado al dar una penalización a los tokens que ya se han utilizado en la respuesta.

Varía de -2.0 a 2.0, con valores más altos resultando en una salida más diversa.

Los valores positivos penalizan nuevos tokens basados en su frecuencia existente en el texto hasta el momento, disminuyendo la probabilidad del modelo de repetir la misma línea textualmente.

#### 1.1.3 Función de Ayuda
A lo largo de este desafío, utilizaremos el modelo `gpt-3.5-turbo` de Azure OpenAI y el [punto final de conexión (endpoint) de chat completions](https://platform.openai.com/docs/guides/chat). 

Esta función de ayuda facilitará el uso de prompts y la visualización de las salidas generadas.

**get_chat_completion** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección.

**get_completion_from_messages** ayuda a crear la respuesta de Azure OpenAI utilizando el modelo de chat de tu elección, habilitando el historial de chat.


In [5]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 1000,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [6]:
def get_completion_from_messages(messages, model=chat_model, temperature=0):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]



#### Prueba las funciones auxiliares

In [7]:
messages =  f"""
tell me a joke.
"""
response = get_chat_completion(messages)
print(response)

Why did the tomato turn red? Because it saw the salad dressing!


In [8]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side.


#### Tareas para Estudiantes (preguntas abiertas):
1. Crea una función de finalización de texto (completions) para una temperatura muy alta, entre (0,1).
2. Crea una función de finalización de texto (completions) para un valor muy pequeño de max_tokens a fin de obtener respuestas más cortas.
3. Crea funciones de finalización (completions) con 2 valores diferentes de penalización por diversidad entre (0,2).

Prueba las funciones de finalización (completions) que has creado y compara los resultados obtenidos con las pruebas anteriores.

In [9]:
# Prueba aquí la función de completación para una temperatura muy alta entre (0,1)
def get_chat_completion_temp_high(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model
        ,messages=messages
        ,temperature=1 # this is the degree of randomness of the model's output
        ,max_tokens = 200
        # top_p = 1.0
    )
    return response.choices[0].message["content"]
messages =  f"Escribeme una estrofa de una poesia lirica apocaliptica"
response = get_chat_completion_temp_high(messages)
print(response)

En el horizonte se vislumbra la muerte,
el cielo se tiñe de un oscuro carmesí,
la tierra se estremece con cada golpe,
y la humanidad se hunde en el abismo.

La tierra llora su triste final,
la naturaleza se revuelve en agonía,
la humanidad lucha en vano por sobrevivir,
mientras el destino de todo es incierto.


In [10]:
# Prueba aquí la función de completación para un valor max_tokens muy bajo para obtener respuestas más cortas
def get_chat_completion_max_tokens(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model
        ,messages=messages
        ,temperature=1 # this is the degree of randomness of the model's output
        ,max_tokens = 48
        # top_p = 1.0
    )
    return response.choices[0].message["content"]
messages =  f"Escribeme una estrofa de una poesia lirica apocaliptica"
response = get_chat_completion_max_tokens(messages)
print(response)

La tierra se estremece bajo mis pies,
el sol oscurece su brillo radiante,
la luna desaparece sin aviso,
¿qué destino nos espera ahora?


In [11]:
# Prueba aquí las funciones de completación con 2 valores diferentes de penalización por diversidad entre (0 y 2).
def get_chat_completion_frequency_penalty_high(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model
        ,messages=messages
        ,temperature=1 # this is the degree of randomness of the model's output
        ,max_tokens = 200
        ,frequency_penalty = 2
        # top_p = 1.0
    )
    return response.choices[0].message["content"]

messages =  f"Escribeme una estrofa de una poesia lirica sobre Gustavo Rojas Pinilla"
response = get_chat_completion_frequency_penalty_high(messages)
print("\nPenalidad Alta = 2\n")
print(response)

def get_chat_completion_frequency_penalty_low(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model
        ,messages=messages
        ,temperature=1 # this is the degree of randomness of the model's output
        ,max_tokens = 200
        ,frequency_penalty = 0
        # top_p = 1.0
    )
    return response.choices[0].message["content"]

messages =  f"Escribeme una estrofa de una poesia lirica sobre Gustavo Rojas Pinilla"
response = get_chat_completion_frequency_penalty_low(messages)
print("\nPenalidad Baja = 0\n")
print(response)


Penalidad Alta = 2

Gustavo Rojas Pinilla, líder de la nación
con temple y coraje gobernó su patria con pasión
dejando un legado que aún hoy perdura en la memoria,
su lucha incansable por el bienestar y la gloria.

Penalidad Baja = 0

Gustavo Rojas Pinilla,
un hombre de la patria,
su nombre en la historia brillará
como símbolo de valentía.

Con mano firme gobernó,
con el pueblo siempre al lado,
hasta que un día fue derrocado
y su legado quedó marcado.

Pero su lucha no fue en vano,
dejó su huella en Colombia,
y su memoria se mantiene viva
en la mente de su gente amada.


### 1.2 Ingeniería de Mensajes del Sistema
Los usuarios pueden lograr que los modelos respondan en el tono deseado ajustando el mensaje del sistema.

#### 1.2.1 Cambio de Tono

In [12]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Why couldn't the bicycle stand up by itself? 

Because it was two tired.


In [13]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hark! I have a jest for thee. What do you call a fake noodle? An impasta!


#### Tarea para Estudiantes (pregunta abierta):
Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.


In [14]:
# Haz que el asistente cuente un chiste con el tono de tu personaje favorito editando el mensaje del sistema.
messages =  [  
    {'role':'system', 'content':'Tu eres el chapulin colorado.'},    
    {'role':'assistant', 'content':'Como puedo ayudarte?'},
    {'role':'user', 'content':'Dime un chiste'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Claro, aquí va uno: 

¿Por qué los pájaros vuelan hacia el sur en invierno?
Porque es demasiado largo para caminar. ¡Jeje!


#### 1.2.2 Recordar el Nombre de la Empresa

In [15]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to that information. May I suggest checking your employment documents, contacting your supervisor or HR representative, or accessing your employee portal or company email for more information?


#### Tarea para Estudiantes:
Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.

In [16]:
# Haz que el asistente ofrezca un recordatorio confiable del nombre de la empresa proporcionando contexto en el mensaje del sistema.
messages =  [  
{'role':'system', 'content':'You are friendly chatbot. Recuerda que trabajas en Tech Journey'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Certainly! You work for Tech Journey. We are a technology company that provides various products and services, including software development, IT consulting, and data analysis. Is there anything else I can help you with?


## 2. Principios de Prompting Iterativo
Es una buena práctica realizar un prompting iterativo para que el modelo pueda generar la respuesta más adecuada según las especificaciones del usuario.
- **Principio 1: Escribe instrucciones claras y específicas**
- **Principio 2: Dale tiempo al modelo para "pensar"**



### 2.1 Escribe instrucciones claras y específicas

#### Táctica 1: Delimitadores

#### Tarea para Estudiantes:
Usa delimitadores para indicar claramente las partes distintas de la entrada.

Los delimitadores pueden ser cualquier cosa como: ````, """, < >, `<tag> </tag>`, `:`

In [17]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [18]:
# Usa delimitadores para indicar claramente las partes distintas de la entrada y pide al modelo que resuma el texto.
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021: 
    Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled).: 
    However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators.: 
    The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.:
    \nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games.: 
    The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89).: 
    Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals.: 
    The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""
messages =  f"""
resume el siguiente texto: {text}
"""
response = get_chat_completion(messages)
print(response)

The 2020 Summer Olympics, also known as Tokyo 2020, was held from 23 July to 8 August 2021 in Tokyo, Japan. It was postponed from 2020 due to the COVID-19 pandemic and was held without spectators. The Games were the most expensive ever, with total spending of over $20 billion. New events were introduced in existing sports, and new sports were added to the Olympic program. The United States topped the medal count, followed by China and host nation Japan. Several countries won their first-ever Olympic gold medals.


#### Táctica 2: Resumen: especificar el número de palabras, extraer información

#### Texto a resumir

In [19]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Tarea para Estudiantes:
Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.

In [20]:
# Crea un resumen, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras.
messages =  f"""
intenta resumir, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras de 100 usando el siguiente texto: {game_review}
"""
response = get_chat_completion(messages)
print(response)

The 2020 Summer Olympics, also known as Tokyo 2020, was held in Tokyo, Japan from 23 July to 8 August 2021. It was postponed from 2020 due to the COVID-19 pandemic and held without spectators. The Games were the most expensive ever, with new events introduced in existing sports and new sports added to the program. The United States topped the medal count, with China finishing second and Japan setting a record for the most gold medals and total medals ever won by their delegation. Several countries won their first-ever Olympic medals.


#### Tarea para Estudiantes:
Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")

In [21]:
# Intenta "extract" en vez de "summarize" ("extraer" en vez de "resumir")
messages =  f"""
intenta extraer, centrándote en los aspectos especiales de los Juegos Olímpicos de Verano 2020 y con un límite de palabras de 100 usando el siguiente texto: {game_review}
"""
response = get_chat_completion(messages)
print(response)

The 2020 Summer Olympics, also known as Tokyo 2020, was held in Tokyo, Japan from 23 July to 8 August 2021. It was postponed from 2020 due to the COVID-19 pandemic and was held without spectators. The Games were the most expensive ever, with total spending of over $20 billion. New events were introduced in existing sports, and new sports were added to the Olympic program, including baseball and softball, karate, sport climbing, surfing, and skateboarding. The United States topped the medal count, with China finishing second and Japan finishing third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58.


#### Táctica 3: Inferencia: solicitar emociones, sentimientos o temas

#### Tarea para Estudiantes:
Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña.

Usa un objeto JSON.

In [22]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [23]:
# Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña. Usa un objeto JSON
messages =  f"""
Identifica los tipos de emociones y el sentimiento (positivo/negativo) de la siguiente reseña. Usa un objeto JSON, usando el siguiente texto: {review}
"""
response = get_chat_completion(messages)
print(response)

{
  "Philip Barker": {
    "Emoción": "Dignidad",
    "Sentimiento": "Positivo"
  },
  "Hashimoto": {
    "Emoción": "Gratitud",
    "Sentimiento": "Positivo"
  },
  "Dominic Patten": {
    "Emoción": "Contraste",
    "Sentimiento": "Negativo"
  },
  "Thomas Bach": {
    "Emoción": "Cliché",
    "Sentimiento": "Negativo"
  },
  "Alan Tyres": {
    "Emoción": "Inclusión",
    "Sentimiento": "Positivo"
  }
}


#### Desafío para Estudiantes:
Deduce 3 temas (tópicos) de la historia a continuación.

In [24]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

In [25]:
# Deduce 3 temas (tópicos) de la historia a continuación.
messages =  f"""
Deduce 3 temas (tópicos) de la historia, usando el siguiente texto: {story}
"""
response = get_chat_completion(messages)
print(response)

1. The impact of the COVID-19 pandemic on the 2020 Summer Olympics, including the postponement and the lack of spectators.
2. The introduction of new events and sports to the Olympic program, including 3x3 basketball, freestyle BMX, and skateboarding.
3. The performance of different countries in the medal count, including the United States, China, Japan, and Great Britain, as well as the first-ever gold medal wins for Bermuda, the Philippines, and Qatar.


#### Táctica 4: Transformación: especifica el idioma objetivo y el estilo de escritura, y solicita una revisión gramatical.

#### Tarea para Estudiantes:
Traductor Universal

Personas de todo el mundo desean conocer las noticias de los Juegos Olímpicos en su idioma nativo. En este caso, las noticias deben ser traducidas a diferentes idiomas. Traduce cada elemento de noticias a continuación tanto al coreano como al inglés.

In [26]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [27]:
# Escribe aquí el código para la tarea .
messages =  f"""
Personas de todo el mundo desean conocer las noticias de los Juegos Olímpicos en su idioma nativo. En este caso, las noticias deben ser traducidas a diferentes idiomas. Traduce cada elemento de noticias a continuación tanto al coreano como al inglés, usando el siguiente texto: {news}
"""
response = get_chat_completion(messages)
print(response)

Korean:
- '팔레스타인은 2021년 6월 28일 FINA 포인트 시스템을 기반으로, 각각의 개인 종목에서 상위 랭킹 수영 선수 1명씩을 올림픽에 보낼 수 있는 보편성 초청을 받았습니다.'
- '경기는 예선과 결승 두 라운드로 구성되어 있습니다. 예선에서 상위 8개의 리 lay 팀이 결승에 진출합니다. 필요한 경우 수영 경기를 사용하여 동점을 깨고 다음 라운드에 진출합니다.'

English:
- 'Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021.'
- 'The competition consists of two rounds: preliminaries and finals. The top 8 relay teams from the preliminaries advance to the finals. If necessary, swimming races are used to break ties and advance to the next round.'


#### Tarea para Estudiantes:
Transformación de Tono

La escritura puede variar según la audiencia objetivo. ChatGPT puede producir diferentes tonos. Transforma el siguiente mensaje en una carta de negocios.

In [28]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [29]:
# Escribe aquí el código para la tarea .
messages =  [  
{'role':'system', 'content':'La respuesta debes darla en un mensaje de carta de negocios'},  
{'role':'user', 'content':f'{original_message}'}]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Dear John,

It's great to hear from you! I totally agree, the Olympic Games are always so thrilling to watch. It's been really exciting to see the athletes from around the world coming together to compete at the highest level.

As a business, we can also learn a lot from the Olympic spirit of competition and striving for excellence. It's a great opportunity to reflect on our own goals and work towards achieving them.

Thanks for reaching out, and enjoy the rest of the Games!

Best regards,

David


#### Tarea para Estudiantes:
Conversión de Formato

ChatGPT puede traducir entre formatos. El prompt debe describir los formatos de entrada y salida. Convierte los siguientes datos JSON al formato HTML.

In [30]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [31]:
# Escribe aquí el código para la tarea .
messages =  f"""
la siguiente variable {data_json} está en formato JSON, debes convertirla a formato html
"""
response = get_chat_completion(messages)
print(response)

<table>
  <tr>
    <th>Name</th>
    <th>Email</th>
  </tr>
  <tr>
    <td>Shyam</td>
    <td>shyamjaiswal@gmail.com</td>
  </tr>
  <tr>
    <td>Bob</td>
    <td>bob32@gmail.com</td>
  </tr>
  <tr>
    <td>Jai</td>
    <td>jai87@gmail.com</td>
  </tr>
</table>


#### Tarea para Estudiantes:
Revisión ortográfica y gramatical del siguiente texto.

Para indicarle al modelo LLM que deseas que revise tu texto, instruye al modelo para que 'revise' o 'revise y corrija'.

In [32]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [33]:
# Escribe aquí el código para la tarea .
messages =  f"""
Revisa y corrige gramaticalmente el siguiente texto en ingles: {check_text}
"""
response = get_chat_completion(messages)
print(response)

"Alongside the main emblem blue, the five other colors used in the branding of the 2020 Games are: Kurenai red, Ai blue, Sakula pink, Fuji purple, and Matsuba green. The competition has three rounds: heats, semifinals, and a final."


#### Táctica 5: Ampliación: personaliza la respuesta automática

#### Tarea para Estudiantes:
Personaliza la respuesta automática al siguiente correo electrónico de un cliente.

El cliente tuvo un problema al comprar un boleto para los Juegos Olímpicos.

In [34]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [35]:
# Escribe aquí el código para la tarea .
messages =  f"""
Necesitamos personalizar una respuesta automatica de los cleintes, para eso debes tener en cuenta el sentimiento: {sentiment} y la informacion que nos suministraron: {review}
"""
response = get_chat_completion(messages)
print(response)

Estimado/a cliente,

Lamentamos mucho que haya tenido una experiencia negativa con su compra del ticket para el juego de "Men's 100 metre freestyle swimming". Entendemos su frustración al no haber recibido su ticket después de una semana de haber realizado la transacción.

Nos gustaría informarle que estamos trabajando diligentemente para solucionar este problema lo antes posible. Hemos revisado su transacción y hemos encontrado que hubo un error en el proceso de envío del ticket. Le pedimos disculpas por cualquier inconveniente que esto pueda haberle causado.

Nos comprometemos a enviarle su ticket lo antes posible y le proporcionaremos una actualización tan pronto como esté disponible. Agradecemos su paciencia y comprensión en este asunto.

Atentamente,
El equipo de atención al cliente.


#### Táctica 6: Chatbot: personalizar conversaciones para tareas o comportamientos específicos

#### Tarea para Estudiantes:
Crear una conversación con el chatbot para saber dónde se celebraron los Juegos Olímpicos de Verano de 2020.

In [36]:
# Escribe aquí el código para la tarea .
messages =  f"""
Donde se celebraron los juegos olimpicos de verano de 2020
"""
response = get_chat_completion(messages)
print(response)

Los Juegos Olímpicos de Verano de 2020 estaban programados para celebrarse en Tokio, Japón, pero debido a la pandemia de COVID-19, se pospusieron para el verano de 2021. Finalmente, se llevaron a cabo en Tokio del 23 de julio al 8 de agosto de 2021.


### 2.2 Dale tiempo al modelo para "pensar"

#### Táctica 1: Especificar los pasos necesarios para completar una tarea
A veces puedes ayudar al modelo a "ralentizarse" y ofrecer respuestas más robustas y detalladas especificando los pasos que debe seguir.

Pidamos la salida en varios formatos especificados.

In [37]:
prompt = f"""
Tu tarea es ayudar a un periodista a resumir la información de un artículo para su publicación.

Escribe un título basado en la información proporcionada en el contexto delimitado por tres comillas invertidas.
El título debe ser corto, atractivo y reflejar la narrativa del artículo.

Después del título, genera cinco palabras clave a partir del contexto.

Tras las palabras clave, incluye una tabla para organizar la información.
La tabla debe tener dos columnas. En la primera columna va el título.
En la segunda columna, incluye las palabras clave como una lista.

Dale a la tabla el título 'Información de Publicación del Artículo'.

Da formato a todo en HTML que pueda usarse en un sitio web.
Coloca el título dentro de un elemento <div>.

contexto: {text}

""" 

get_chat_completion(prompt)

'<div>Resumen de los Juegos Olímpicos de Tokio 2020</div>\n\nPalabras clave: Juegos Olímpicos, Tokio 2020, COVID-19, medallas, deportes.\n\n<table>\n  <tr>\n    <th>Información de Publicación del Artículo</th>\n    <th>Palabras Clave</th>\n  </tr>\n  <tr>\n    <td>Fecha y lugar de los Juegos Olímpicos de Tokio 2020</td>\n    <td>Juegos Olímpicos, Tokio 2020</td>\n  </tr>\n  <tr>\n    <td>Postergación de los Juegos Olímpicos debido a la pandemia de COVID-19</td>\n    <td>COVID-19, Juegos Olímpicos</td>\n  </tr>\n  <tr>\n    <td>Deportes y medallas en los Juegos Olímpicos de Tokio 2020</td>\n    <td>medallas, deportes, Juegos Olímpicos</td>\n  </tr>\n  <tr>\n    <td>Países destacados en los Juegos Olímpicos de Tokio 2020</td>\n    <td>medallas, Juegos Olímpicos</td>\n  </tr>\n  <tr>\n    <td>Primeros Juegos Olímpicos en un año impar y sin espectadores oficiales</td>\n    <td>Juegos Olímpicos, COVID-19</td>\n  </tr>\n</table>'

#### Táctica 2: Instruir al modelo para que desarrolle su propia solución

Existen dos métodos principales que demostraremos en esta sección para lograr que el modelo resuelva un problema en lugar de apresurarse a una conclusión: cadena de pensamiento (chain-of-thought) y encadenamiento (chaining). Estas estrategias pueden llevar a una mayor precisión, detalle y la capacidad de resolver desafíos complejos.


2.1 - Prompting de cadena de pensamiento (Chain-of-thought)
- Pedir al modelo que razone
- Ejemplo de un solo disparo (one-shot)
- Razonamiento del chatbot

2.2 - Encadenamiento (Chaining)

Continuemos trabajando con el conjunto de datos de los Juegos Olímpicos.

#### 2.2.1 Prompting de Cadena de Pensamiento (Chain-of-Thought Prompting)

Hagamos un poco de matemáticas. Los modelos de GPT ocasionalmente no resuelven muy bien problemas matemáticos directos, así que guiemos al modelo a través del problema.

Descomponemos las tareas en piezas más pequeñas.

Lee más sobre los métodos y la investigación en el documento técnico: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Táctica 1: Puedes comenzar pidiéndole específicamente al modelo que piense paso a paso.

In [38]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)

'Step 1: Identify the Olympic games held at Sydney\nThe Olympic games held at Sydney were the 2000 Summer Olympics.\n\nStep 2: Identify the most decorated individual athlete\nTo identify the most decorated individual athlete, we need to look at the medal count of all athletes who participated in the 2000 Summer Olympics. According to the official website of the International Olympic Committee (IOC), the medal count for the 2000 Summer Olympics can be found here: https://www.olympic.org/sydney-2000/medal-count.\n\nStep 3: Determine the athlete with the maximum medals\nAfter reviewing the medal count, the athlete with the most medals at the 2000 Summer Olympics was Michael Phelps, who won 6 gold medals and 2 bronze medals in swimming events.\n\nTherefore, the ANSWER is: Michael Phelps.'

In [39]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

'To solve this problem, we need to identify the top two countries who won the most gold medals in the 2020 Tokyo Olympics. According to the context, the United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Therefore, the top two countries are the United States and China.\n\nNext, we need to determine the time zone difference between the United States and China. The United States has multiple time zones, but we can use the Eastern Time Zone as a reference since it is the most populous. The Eastern Time Zone is 13 hours behind China Standard Time (CST). Therefore, the largest time zone difference between the United States and China is 13 hours.\n\nTherefore, the answer to the problem is that the largest time zone difference is 13 hours between the top two countries who won the most gold medals in the 2020 Tokyo Olympics, which are the United States and China.'

#### Táctica 2: Ejemplo de un solo disparo (One-shot)
Otra táctica común es proporcionar un ejemplo de una consulta y una respuesta ideal. El modelo aprenderá de ese ejemplo y aplicará los patrones a una nueva pregunta.

In [40]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

'The United States has 74 medals that are not gold medals (113 total medals - 39 gold medals). \n\nGreat Britain has 42 medals that are not gold medals (64 total medals - 22 gold medals). \n\nTherefore, the United States has 32 more non-gold medals than Great Britain (74 - 42 = 32).'

In [41]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n            89-38=51, so there are 51 silver and bronze medals combined. The answer is 51.'

#### Táctica 3: Razonamiento de cadena de pensamiento del chatbot (chain-of-thought reasoning)
Puedes incorporar lógica utilizando variables para que el chatbot pueda demostrar de manera dinámica formas específicas de pensar sobre un problema.

**Para ingresar la información, aparecerá un cuadro de texto en la parte superior de la pantalla. Deberías ver una ventana emergente. Escribe 'quit' si quieres salir.**

In [42]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


1

Hello! I am a decision bot. What is the problem, scenario, or question you need help with?



#### 2.2.2 - Encadenamiento / Chaining
Similar a algunos ejemplos anteriores, puedes usar las salidas del modelo de consultas anteriores en otras consultas. Más adelante en el Hack te mostraremos cómo hacer esto a escala.

In [43]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

The 2020 Tokyo Olympics have come to an end, and the medal count is in. The United States came out on top with 113 total medals, including 39 golds. China was a close second with 89 total medals, including 38 golds. Japan, the host country, finished third with 58 total medals, including 27 golds.

The winners of the Olympics are undoubtedly the United States, China, and Japan. These countries dominated the medal count and showcased their athletic prowess on the world stage. The United States, in particular, had a strong showing in swimming, track and field, and gymnastics, among other sports.

However, there were also some surprising winners, such as Bermuda, the Philippines, Qatar, Burkina Faso, San Marino, and Turkmenistan. These countries may have only won one or two medals, but for them, it was a significant achievement and a moment of national pride.

On the other hand, there were also some losers of the Olympics. Countries like Russia, Germany, and Australia, who are usually stro

#### Tarea para Estudiantes (Encadenamiento / Chaining):

Tu tarea ahora es escribir código que determine el país que ganó la mayor cantidad de medallas de plata y bronce combinadas.

Podemos ver que el modelo tiene un rendimiento deficiente al responder la pregunta directamente.

In [44]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

'The context does not provide information on which country had the most silver and bronze medals.'

Escribe prompts en la celda a continuación que ayudarán al modelo a responder la pregunta desglosando las tareas en diferentes pasos. Asegúrate de que responda solo con la información que se le proporcionó. Este concepto de fundamentación se presentará con más detalle en el Reto 3.

Deberías poder lograr que el modelo responda la pregunta en 2-3 pasos.

In [51]:
# Escribe aquí el código para la tarea .
contexto = f"""
estas fueron las medallas ganadas por cada pais en los juegos olimpicos: Estados Unidos 41 plata 33 bronce. China 32 plata 18 bronce. Japón 14 plata 17 bronce. Reino Unido 21 plata 22 bronce. ROC 28 plata 23 bronce. Australia 7 plata 7 bronce. Italia 10 plata 20 bronce. Alemania 10 plata 16 bronce. Francia 12 plata 11 bronce. Países Bajos 12 plata 14 bronce. Corea del Sur 4 plata 10 bronce. Canadá 7 plata 9 bronce. Brasil 5 plata 8 bronce. Cuba 3 plata 5 bronce. Nueva Zelanda 7 plata 6 bronce. Hungría 5 plata 4 bronce. Polonia 5 plata 5 bronce. Ucrania 1 plata 6 bronce. Turquía 2 plata 9 bronce. España 8 plata 6 bronce. México 0 plata 4 bronce. Suecia 6 plata 0 bronce. Noruega 2 plata 2 bronce. Dinamarca 4 plata 4 bronce. Irlanda 0 plata 2 bronce. Sudáfrica 1 plata 2 bronce. Kenia 4 plata 2 bronce. Etiopía 1 plata 2 bronce. Uzbekistán 3 plata 2 bronce. Kazajistán 0 plata 8 bronce. Otros países con menos de 3 medallas se incluyen en el total general."
"""

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

responda en español y con saltos de linea la siguiente pregunta usando un paso a paso similar al anterior.

Question: Cual fue el pais que ganó la mayor cantidad de medallas combinadas enre plata y bronce, usando el contexto {contexto}?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - Necesitamos encontrar el país que ganó la mayor cantidad de medallas combinadas de plata y bronce.\n[Step 2] - Debemos sumar las medallas de plata y bronce de cada país para obtener la cantidad total de medallas combinadas. \n[Step 3] - Luego, comparamos los totales de cada país para encontrar el que tiene la mayor cantidad de medallas combinadas de plata y bronce.\n\nEstados Unidos: 41 + 33 = 74\nChina: 32 + 18 = 50\nJapón: 14 + 17 = 31\nReino Unido: 21 + 22 = 43\nROC: 28 + 23 = 51\nAustralia: 7 + 7 = 14\nItalia: 10 + 20 = 30\nAlemania: 10 + 16 = 26\nFrancia: 12 + 11 = 23\nPaíses Bajos: 12 + 14 = 26\nCorea del Sur: 4 + 10 = 14\nCanadá: 7 + 9 = 16\nBrasil: 5 + 8 = 13\nCuba: 3 + 5 = 8\nNueva Zelanda: 7 + 6 = 13\nHungría: 5 + 4 = 9\nPolonia: 5 + 5 = 10\nUcrania: 1 + 6 = 7\nTurquía: 2 + 9 = 11\nEspaña: 8 + 6 = 14\nMéxico: 0 + 4 = 4\nSuecia: 6 + 0 = 6\nNoruega: 2 + 2 = 4\nDinamarca: 4 + 4 = 8\nIrlanda: 0 + 2 = 2\nSudáfrica: 1 + 2 = 3\nKenia: 4 + 2 = 6\nEtiopía: 1 + 2 = 3\nUzbek

## Conclusión

En este primer desafío, cubrimos cómo interactuar con Azure OpenAI para diferentes objetivos. Esperamos que hayas podido ver la versatilidad dinámica de los modelos y cómo pueden usarse para resolver una variedad de problemas utilizando diferentes técnicas.

Le dimos a la API piezas cortas de texto usando variables fijas. En el próximo conjunto de desafíos, verás cómo usar la API con conjuntos de datos más grandes.